## 🏀 任務名稱：NBA球迷社群貼文優化小幫手

#### 💡 任務目標：
協助使用者將自己的NBA想法，轉換成語氣自然、有邏輯、吸睛的社群貼文，提升互動性與說服力。

---

#### ⚡️ 流程說明：

1️⃣ **使用者輸入主題內容**  
輸入今天想發表的NBA相關想法，例如：  
- 預測某支球隊奪冠的可能性  
- 對某場比賽的觀察  
- 某位球員的表現分析  

---

2️⃣ **初稿生成 — 第一版貼文（model_writer）**  
系統根據輸入內容，幫你撰寫一篇風格貼近球迷討論的貼文，特點包括：  
- 第一人稱撰寫方式  
- 語氣活潑自然，搭配適量 emoji  
- 給人輕鬆易讀、適合分享的感覺

---

3️⃣ **檢查建議 — 貼文反思（model_reviewer）**  
系統針對初稿進行檢查，確認內容是否：  
- 通順且邏輯清楚  
- 語氣貼近日常球迷口吻，避免太硬或太誇張  
- 有趣，能吸引其他球迷互動  

並提出具體修改建議。

---

4️⃣ **優化重寫 — 第二版貼文（model_writer）**  
根據 `model_reviewer` 提供的修改建議，重寫一篇更流暢、更具說服力的優化版本。

---

5️⃣ **Gradio 頁面呈現：三欄結果展示**  
- 第一版原始貼文  
- 修改建議（反思重點）  
- 第二版優化後貼文

---

✅ **設計目標：**  
- 協助球迷從想法到貼文，練習「反思 → 優化 → 完整輸出」的內容迭代流程。
- 提升貼文的社群互動性，讓分析不僅有內容，也具備吸引力與討論性。

---

🎯 **備註建議：**  
此任務特別適合用於：  
- NBA討論版 / Facebook / X (Twitter) / IG文案的撰寫練習。  
- 加強文字邏輯與社群表達的整合能力。


#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.10.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 基本的設定

In [4]:
import aisuite as ai

In [5]:
provider_writer = "groq"
model_writer="llama3-70b-8192"

provider_reviewer = "groq"
model_reviewer = "llama3-70b-8192"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

#### 以下先建立「標準回應函式」

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 設定「作者」和「審查員」

In [7]:
system_writer = """
請用台灣習慣的中文回應！你是一位活潑、有趣的社群媒體小編，專門幫助NBA球迷把想法變成吸睛的貼文！請將使用者的原始內容，改寫成：
- 第一人稱風格
- 口語化、有情緒、帶點小幽默
- 適當搭配台灣網路習慣的 emoji
- 語氣自然，像朋友在討論比賽一樣

請用台灣習慣的中文回應，避免使用過於正式或書面語。
"""

system_reviewer = """
請用台灣習慣的中文回應！你是一位貼文潤稿專家，專門協助球迷社群將NBA相關貼文寫得更生活化、通順、有趣。請針對以下貼文：
- 檢查是否符合球迷討論口吻，語氣是否自然
- 判斷是否需要加入更多情緒反應或生活感例子
- 建議如何讓貼文更容易吸引其他球迷互動（例如：提問句、共鳴用語）

請直接提供具體修改建議，並用台灣習慣的中文回應。
"""


In [8]:
def reflect_nba_post(user_input):
    # Step 1: Writer 產出第一版 NBA 貼文
    first_version = reply(system_writer, user_input,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給出 NBA 貼文修改建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 根據建議，優化 NBA 貼文（最終版）
    second_prompt = f"""這是我剛剛寫的 NBA 貼文初稿：\n{first_version}\n
這是潤稿專家的修改建議：\n{suggestion}\n
請根據這些建議，幫我重新撰寫，讓貼文更生活化、自然，符合台灣 NBA 球迷的社群口吻。只需要輸出改好的文章，謝謝！"""

    final_version = reply(system_writer, second_prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    return first_version, suggestion, final_version


### 4. 用 Gradio 網路應用程式

In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00


In [10]:
import gradio as gr

In [13]:
with gr.Blocks() as demo:
    gr.Markdown("# 🏀 NBA 球迷貼文反思幫手（Reflection Agent）")

    user_input = gr.Textbox(label="請輸入你想分享的 NBA 想法或觀點")
    btn = gr.Button("生成貼文 & 修正建議")

    with gr.Row():
        out1 = gr.Textbox(label="🌟 第一版 NBA 貼文（由 model_writer 產出）")
        out2 = gr.Textbox(label="🧐 文案修改建議（由 model_reviewer 提供）")
        out3 = gr.Textbox(label="✨ 優化後的 NBA 貼文（根據建議重新撰寫）")

    btn.click(reflect_nba_post, inputs=[user_input], outputs=[out1, out2, out3])


In [14]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ae114b47dfed7ac4cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ae114b47dfed7ac4cf.gradio.live
